# KG-Hub Tutorial 5 - Graph ML case study

##### Here we describe a more detailed graph ML case study using KG-Hub graph data. The intention is to show how the various features of KG-Hub and GRAPE can be used to extract knowledge from KGs using graph ML.

This workflow will demonstrate how to use KG-Hub and GRAPE for:
- Getting the graph
    - getting a stable, versioned graph data from KG-Hub for a particular KG project
- Loading the graph
    - loading the graph in GRAPE and getting a summary of the graph and its topology
- Examining the graph
    - getting a sense of the graph topology
- Making node embeddings and evaluate them
    - producing node embeddings, and evaluating them with detailed metrics provided by GRAPE
- Using cosine similarity to rank drugs for relevance to COVID-19

This will demonstrate several useful features of KG-Hub and GRAPE in graph ML workflows:
- Pinning of analyses to versioned builds of KG projects (here the 20200925 build of KG-COVID-19, using stable, permanent URLs
- Tight integration of KG-Hub graphs with GRAPE - KG-Hub data can be read directly into GRAPE
- Using GRAPE to produce a very detailed analysis of KGs from KG-Hub, summarizing the data and graph topology in seconds
- Quickly producing node embeddings and examining the features of these node embeddings (see tSNE plots below)
- Using these node embeddings to rank the relevance of drugs in the KG to a node of interest (here, SARS-CoV-2)

## Getting the graph

First, install GraPE and a variety of other dependencies with `pip`:

In [ ]:
!pip install -qU grape

We will have a look here to see what builds of this KG are available: `https://kg-hub.berkeleybop.io/kg-covid-19/`

We will pin our analysis to one of the earlier builds (20200925) of KG-COVID-19. This project has evolved a lot since Sept 2020, but this will give us only data that was available early on in the pandemic.

In [ ]:
!wget https://kg-hub.berkeleybop.io/kg-covid-19/20200925/kg-covid-19.tar.gz
!tar -xvzf kg-covid-19.tar.gz

## Loading the graph

In [ ]:
from grape import Graph
g = Graph.from_csv(
  directed=False, # This is a directed graph - we're treating it as undirected to make it easier to examine paths.
  node_path='merged-kg_nodes.tsv',
  edge_path='merged-kg_edges.tsv',
  verbose=True,
  nodes_column='id',
  node_list_node_types_column='category',
  default_node_type='biolink:NamedThing',
  sources_column='subject',
  destinations_column='object',
  edge_list_edge_types_column='relation',
  name="KG-COVID-19 version 20200925"
)

We will select only the largest connected component from the KG to analyze:

In [ ]:
g = g.remove_components(top_k_components=1)

## Examine the graph

Before doing any graph ML, it is usually a good idea to take a look at the graph and get a sense of the data and its topology

GRAPE makes this very easy - just type the name of graph (here `g`) and GRAPE will return a very detailed summary of the graph:

In [ ]:
g

This summary provides a very nice overview of the contents and the topology of our KG. 

The two highest degree nodes are the MESH terms for viruses and humans, which makes sense given the subject of this KG. The most common node type is `biolink:Publication`, owing to our ingest of the Scibite-CORD19 publication dataset. We see some topological oddities of which we should be aware. These include a chain of 14 nodes, about 11k isomorphic node groups (members of these groups will have the same node embeddings for random walk-based embedding methods), and dendritic trees/stars/tendril stars (which will bias embeddings for nodes contained in these dendritic oddities, and will also prevent edges from being selected from 

These oddities can confuse and bias graph ML models, but since we will be using our model to rank interesting drugs that might be followed up on with other work to confirm/corroborate each drug, this is probably okay. GRAPE has methods such as these to remove these oddities if need be:

* `remove_tendrils`
* `remove_dendritic_trees`

## Make node embeddings

Now we will make node embeddings. We are spoiled for choice here - there are many different models to choose from:

In [ ]:
from grape import get_available_models_for_node_embedding
get_available_models_for_node_embedding()

We will use first order Deep Walk SkipGram embeddings, which will take an hour or two, but will capture the topology well.

In [ ]:
from grape.embedders import DeepWalkSkipGramEnsmallen
embeddings = DeepWalkSkipGramEnsmallen().fit_transform(g)

Now let's get a sense of how well we have embedded the nodes. GRAPE provides a very detailed summary of embeddings with just this one command:

In [ ]:
from grape import GraphVisualizer
GraphVisualizer(g).fit_and_plot_all(embeddings)

Above is a detailed analysis of the embeddings based on tSNE transformation of the embeddings, automatically provided by GRAPE. 

Of particular interest to us: in panels B and C we observe fairly good separation between node types and node sources, respectively. This suggests that the embeddings contain information about what sort of entities they are, and implies that they will be useful for downstream tasks such as node classification.

Note also panels D and M, which show analyses of edge embeddings (generated by combining the embeddings of the two nodes involved in the edge). We observe fairly good separation between edges embeddings that actually exist in the graph (orange) and randomly selected 'negative' edges that do not exist in the graph (blue) (note that the y-axis is log-scaled). Again, this suggests that these embeddings will be useful for downstream tasks such as link prediction. 

## Use cosine similarity to rank drugs for relevance to COVID-19

Now let's do an experiment - let's rank drugs by their closeness to SARS-Cov-2, as measured by the cosine similarity between the embedding representing each drug and the embedding representing SARS-Cov-2. We can use this similarity to see if this provides information about drugs that may have a relationship, known or unknown, with COVID-19. 

Here's the node representing SARS-CoV-2:

In [ ]:
sars_cov_2_curie = 'NCBITaxon:227859'

Now let's get all the nodes representing CHEMBL drugs of interest:

In [ ]:
chembl_drugs = [k for (k, v) in g.get_nodes_mapping().items() if k.startswith('CHEMBL.COMPOUND')]

Now make a simple method `get_cosine_sim()` to calculate cosine similarity between SARS-CoV-2 and each CHEMBL drug:

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy import spatial

def get_cosine_sim(sars_cov_2_curie: str, drug_curies: list, embeddings: pd.DataFrame, drug_names: pd.DataFrame) ->  np.ndarray:
    cosine_sim = np.full(len(drug_curies), -1000.0000)
    
    sars_cov_2_embedding = embeddings.loc[sars_cov_2_curie]

    for i, drug_curie in tqdm(enumerate(drug_curies)):
        cosine_sim[i] = 1 - spatial.distance.cosine(sars_cov_2_embedding, 
                                                    embeddings.loc[drug_curie]
                                                   ).flatten()[0]
    df = pd.DataFrame({'drug': drug_curies, 'cosine_sim': cosine_sim})
    
    
    df = pd.merge(df, drug_names, left_on='drug', right_on='id', how='left')
    
    return df.sort_values(by='cosine_sim',axis=0, ascending=False)

In [ ]:
drug_names = pd.read_csv('merged-kg_nodes.tsv', sep='\t', on_bad_lines='skip')[['id', 'name']]
cosine_sims = get_cosine_sim(sars_cov_2_curie, chembl_drugs, embeddings.get_node_embedding_from_index(0), drug_names)

pd.set_option('display.max_rows', 5)
cosine_sims.head(5)

Notice that two antiviral drugs (ribavirin and oseltamivir) appear in the top five ranked drugs. 
Also notice that chloroquine, a malaria drug that was investigated early on in the pandemic for 
COVID-19 treatment but found to not be effective, is in the top five ranked drugs too. This 
underscores the need for subsequent research and validation of all results from ML experiments!
However, this simple experiment indicates that embeddings are providing a ranking of drugs that
have some relationship to COVID-19/SARS-CoV-2.

In [ ]:
cosine_sims.to_csv('cosine_sims.tsv', sep='\t')